In [3]:
import time
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import seaborn as sbn
from scipy import stats

In [5]:
firefile = './output/cleaned_formatted_fire_data.csv'
df = pd.read_csv(firefile, parse_dates=True, infer_datetime_format=True,skip_blank_lines=True, encoding='UTF-8')
df.head()

,OBJECTID,address,address2,apt_room,arrive_date_time,cleared_date_time,dispatch_date_time,exposure,incident_number,incident_type,incident_type_description,platoon,station,Longitude,Latitude
0,474768,"1216 BEVERLY DR RALEIGH, NC 27601",NaN,,2007-08-21 22:52:00,2007-08-21 22:58:00,2007-08-21 22:47:00,0,07-0023051,444,Power line down,A,12,-78.595422,35.761213
1,474769,"8332 SHILOH CREEK CT RALEIGH, NC 27603",NaN,,2013-10-19 21:52:00,2013-10-19 21:59:00,2013-10-19 21:47:00,0,13-0030918,150,Outside rubbish fire Other,B,22,-78.537320,35.891622
2,474770,"7513 IDOLBROOK LN RALEIGH, NC 27615",NaN,,2014-02-14 21:49:00,2014-02-14 21:51:00,2014-02-14 21:46:00,0,14-0004846,510,Person in distress Other,A,4,-78.647631,35.878186
3,474771,"1205 S BLOODWORTH ST RALEIGH, NC 27601",NaN,,2012-04-29 05:17:00,2012-04-29 05:38:00,2012-04-29 05:16:00,0,12-0011779,440,Electrical wiring/equipment problem Other,A,1,-78.633977,35.765388
4,474772,"7628 VALVIEW ST RALEIGH, NC 27601",NaN,,2012-04-28 23:46:00,2012-04-28 23:52:00,2012-04-28 23:43:00,0,12-9010131,561,Unauthorized burning,A,18,-78.691396,35.884447
